In [1]:
!pip install pandas altair altair_data_server

In [63]:
import pandas
import altair

altair.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

In [64]:
data = pandas.read_json('use_numpy_api.jsonl', lines=True)
data

,function,caller_args,caller_kwargs,signature_available,function_arguments,function_var_positional,function_var_keyword
0,numpy.arange,[10],{},False,None,None,None
1,_operator.pos,"[{'__tp': 'numpy.ndarray', '__v': {'shape': [1...",{},True,"{'a': {'__tp': 'numpy.ndarray', '__v': {'shape...",[],{}
2,_operator.neg,"[{'__tp': 'numpy.ndarray', '__v': {'shape': [1...",{},True,"{'a': {'__tp': 'numpy.ndarray', '__v': {'shape...",[],{}
3,_operator.invert,"[{'__tp': 'numpy.ndarray', '__v': {'shape': [1...",{},True,"{'a': {'__tp': 'numpy.ndarray', '__v': {'shape...",[],{}
4,_operator.not_,"[{'__tp': 'numpy.ndarray', '__v': {'shape': [1...",{},True,"{'a': {'__tp': 'numpy.ndarray', '__v': {'shape...",[],{}
...,...,...,...,...,...,...,...
66,builtins.getattr,"[{'__tp': 'numpy.ndarray', '__v': {'shape': [1...",{},False,None,None,None
67,numpy.ndarray.reshape,"[{'__tp': 'numpy.ndarray', '__v': {'shape': [1...",{},False,None,None,None
68,numpy.column_stack,"[[{'__tp': 'numpy.ndarray', '__v': {'shape': [...",{},True,"{'tup': [{'__tp': 'numpy.ndarray', '__v': {'sh...",[],{}
69,builtins.getattr,"[{'__tp': 'builtins.module', '__v': 'numpy'}, ...",{},False,None,None,None


In [65]:
altair.Chart(data).mark_bar().encode(
    y=altair.Y('function', sort='-x'),
    x='count()'
)

alt.Chart(...)

In [66]:
def get_types(name):
    return pandas.DataFrame.from_records(list(data[data.function == name]['caller_args'].to_dict().values()))

In [67]:
slice(1, 2, 3).start

1

In [72]:
get_types('_operator.getitem')

,0,1
0,"{'__tp': 'numpy.ndarray', '__v': {'shape': [10...",0
1,"{'__tp': 'numpy.ndarray', '__v': {'shape': [10...","{'__tp': 'numpy.ndarray', '__v': {'shape': [10..."
2,"{'__tp': 'numpy.ndarray', '__v': {'shape': [10...",0
3,"{'__tp': 'numpy.ndarray', '__v': {'shape': [10...","{'__tp': 'builtins.slice', '__v': [1, 2, None]}"


In [69]:
get_types('numpy.zeros')

,0
0,"[4, 5]"


In [70]:
get_types('numpy.linspace').head()

,0,1,2
0,0,2,100
